**Preprocess watersheds**

Adrian Wiegman

2023-07-11

-------

## Description

This notebook turns raw data from LiDAR DEM, NHD flowlines, and polygons of cranberry bogs, into a layer of polygons representing the topographic catchments draining into each cranberry bog in southeast Massachusetts. 

Even though the lidar data as been hydro flattened/enforced, there are still a number of flowpaths that are not detected underneath highways (e.g. interstate I-495, I-195)

The solution is to "burn" in stream flowlines from the national hydrography dataset. This is done by putting a buffer around the stream flowline network, then assigning an arbitary large value to the stream network polygon, then converting to raster and subtracting the stream elevations from the DEM. 

Once D8 flow direction and flow accumulation rasters have been made the primary objectives can be completed. 

Data Sources:

1. Mass GIS Lidar DEM (1ft vertical resolution, ~1m horizontal resolution)
2. USGS National Hydrography Dataset
- Flow lines
3. Cranberry bogs layer

Steps of Processing: 

1. Prepare Lidar
   - 1M Lidar Elevation -> Clip to study area -> 
   - Resample to 10m resolution using aggregate minimum cell value
   - Fill Sinks 

2. Prepare Flowlines
   - combine flowlines into one layer
   - dissolve flowlines
   - buffer flowlines to 3x the resolution of processed lidar. 
       - buffering width of 15m 
   - convert to raster
3. Burn in flow lines
    - assign a value of -100 feet to flow lines
    - add flowlines to existing elevation (subtract 100 feet)
    
4. fill sinks (again)
5. D8 Flow Direction 
6. Flow Accumulation
7. Generate Bog Pour Points
    - find maximum flow accumulation value inside each bog.
    - generate a point at each bog maximum value.
    - identity to get cranberry bog attributes at each point.
8. Delineate Basins for each point
    - with the bog cranberry points loop through each point
        - delineate watershed using the bog pour point and the D8 flow direction
        - save the output to a temp file with the feature ID ('FID') of the cranberry bog. 
    - merge all cranberry bog basins to one polygon layer, containing the FID of the cranberry bog. 

## Setup Environment

In [4]:
# iphython options
# delete variables in workspace
%reset -f
#places plots inline
%matplotlib inline
#automatically reloads modules if they are changed
%load_ext autoreload 
%autoreload 2
# this codeblock sets up the environment from jupyter notebooks
setup_notebook = "C:/Users/Adrian.Wiegman/Documents/GitHub/Wiegman_USDA_ARS/MEP/_Setup.ipynb"
%run $setup_notebook # magic command to run the notebook 

***
loading python modules...

  `module_list` contains names of all loaded modules

...module loading complete

***
loading user defined functions...

type `fn_`+TAB to for autocomplete suggestions

 the object `def_list` contains user defined function names:
   fn_get_info
   fn_arcgis_table_to_df
   fn_arcgis_table_to_np_to_pd_df
   fn_run_script_w_propy_bat
   fn_try_mkdir
   fn_hello
   fn_recursive_glob_search
   fn_regex_search_replace
   fn_regex_search_0
   fn_arcpy_table_to_excel
   fn_agg_sum_df_on_group
   fn_add_prefix_suffix_to_selected_cols
   fn_calc_pct_cover_within_groups
   fn_buildWhereClauseFromList

 use ??{insert fn name} to inspect
 for example running `??fn_get_info` returns:
Signature: fn_get_info(name='fn_get_info')
Source:   
def fn_get_info(name='fn_get_info'):
    '''
    returns the source information about a given function name
    '''
    get_ipython().run_line_magic('pinfo2', '$name')
File:      c:\workspace\geodata\mep\<ipython-input-1-28f96385f735>
T

## 1. Prepare lidar data

In [ ]:
# 1. Clip Lidar
_i = r"C:\Workspace\Geodata\Massachusetts\LiDAR_DEM\LiDAR_DEM.gdb\LiDAR_DEM_INT_16bit"
mask = r"C:\Workspace\Geodata\MEP\outputs\DWMWPP_WATERSHEDS_SE.shp"
_o = r"C:\Workspace\Geodata\MEP\Default.gdb\LidExtr"
out_raster = arcpy.sa.ExtractByMask(
    in_raster=_i
    in_mask_data="DWMWPP_WATERSHEDS_SE",
    extraction_area="INSIDE",
    analysis_extent='-7945684.04575117 5047354.31786836 -7782812.89626286 5200955.98507201 PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]'
)
out_raster.save(_o)

In [ ]:
# Aggregate data to 10m resolution using minimum cell value
_i = "LidExtr"
_o = "LidAg10m"
out_raster = arcpy.sa.Aggregate(
    in_raster=_i,
    cell_factor=10,
    aggregation_type="MINIMUM",
    extent_handling="EXPAND",
    ignore_nodata="DATA"
)
out_raster.save(_o)

In [10]:
# 3. Fill sinks in 10m resolution lidar
_i = r"C:\Workspace\Geodata\MEP\Default.gdb\LidAg10m_Burn"
_o = "LidAg10m_Fill"
fill = arcpy.sa.Fill(
    in_surface_raster=_i,
    z_limit=3.28 # maximum sink depth = 1 meter
)
fill.save(_o)

## 2. Prepare NHD flow lines

In [ ]:
# merge flow lines
arcpy.management.CopyFeatures(
    in_features=r"C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\NHDFlowline_1",
    config_keyword="",
    spatial_grid_1=None,
    spatial_grid_2=None,
    spatial_grid_3=None
)
arcpy.management.CopyFeatures(
    in_features=r"C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\NHDFlowline_2",
    config_keyword="",
    spatial_grid_1=None,
    spatial_grid_2=None,
    spatial_grid_3=None
)
arcpy.management.Merge(
    inputs="NHDFlowline_1;NHDFlowline_2",
    output=r"C:\Workspace\Geodata\MEP\Default.gdb\NHDFlowlines",
    field_mappings=r'permanent_identifier "Permanent_Identifier" true false false 40 Text 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,permanent_identifier,0,40,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,permanent_identifier,0,40;fdate "FDate" true false false 8 Date 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,fdate,-1,-1,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,fdate,-1,-1;resolution "Resolution" true false false 4 Long 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,resolution,-1,-1,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,resolution,-1,-1;gnis_id "GNIS_ID" true true false 10 Text 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,gnis_id,0,10,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,gnis_id,0,10;gnis_name "GNIS_Name" true true false 65 Text 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,gnis_name,0,65,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,gnis_name,0,65;lengthkm "LengthKm" true true false 8 Double 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,lengthkm,-1,-1,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,lengthkm,-1,-1;reachcode "ReachCode" true true false 14 Text 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,reachcode,0,14,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,reachcode,0,14;flowdir "FlowDir" true false false 4 Long 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,flowdir,-1,-1,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,flowdir,-1,-1;wbarea_permanent_identifier "WBArea_Permanent_Identifier" true true false 40 Text 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,wbarea_permanent_identifier,0,40,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,wbarea_permanent_identifier,0,40;ftype "FType" true false false 4 Long 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,ftype,-1,-1,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,ftype,-1,-1;fcode "FCode" true true false 4 Long 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,fcode,-1,-1,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,fcode,-1,-1;mainpath "MainPath" true false false 4 Long 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,mainpath,-1,-1,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,mainpath,-1,-1;innetwork "InNetwork" true true false 4 Long 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,innetwork,-1,-1,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,innetwork,-1,-1;visibilityfilter "VisibilityFilter" true false false 4 Long 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,visibilityfilter,-1,-1,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,visibilityfilter,-1,-1;SHAPE_Length "SHAPE_Length" false true true 8 Double 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,SHAPE_Length,-1,-1,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,SHAPE_Length,-1,-1;Enabled "Enabled" true true true 2 Short 0 0,First,#,C:\Workspace\Geodata\Massachusetts\NHD_H_01090002_HU8_GDB\NHD_H_01090002_HU8_GDB.gdb\Hydrography\NHDFlowline,Enabled,-1,-1,C:\Workspace\Geodata\Massachusetts\NHD_H_01090004_HU8_GDB\NHD_H_01090004_HU8_GDB.gdb\Hydrography\NHDFlowline,Enabled,-1,-1',
    add_source="ADD_SOURCE_INFO"
)

arcpy.management.Dissolve(
    in_features=r"C:\Workspace\Geodata\MEP\Default.gdb\NHDFlowlines",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\NHDFlowlines_Dissolve",
    dissolve_field="OBJECTID",
    statistics_fields=None,
    multi_part="MULTI_PART",
    unsplit_lines="UNSPLIT_LINES",
    concatenation_separator=""
)
# buffer the flow lines
arcpy.analysis.Buffer(
    in_features=r"C:\Workspace\Geodata\MEP\Default.gdb\NHDFlowlines_Dissolve",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\NHDFlowlines_Dissolve_Buffer",
    buffer_distance_or_field="15 Meters",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="NONE",
    dissolve_field=None,
    method="PLANAR"
)

## 3. Burn NHD flow lines into lidar DEM

In [24]:
# Add field elevation (100 feet)
arcpy.management.CalculateField(
    in_table="NHDFlowlines_Dissolve_Buffer",
    field="Burn",
    expression="-100", # elevation to subtract
    expression_type="PYTHON3",
    code_block="",
    field_type="FLOAT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'NHDFlowlines_Dissolve_Buffer'>

In [25]:
# Feature to raster with same cell size as lidar   
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]', snapRaster=r"Lidar\Mass GIS\LidAg10m_Fill"):
    arcpy.conversion.FeatureToRaster(
        in_features="NHDFlowlines_Dissolve_Buffer",
        field="Burn",
        out_raster=r"C:\Workspace\Geodata\MEP\Default.gdb\BufferedFlowlinesRaster",
        cell_size=r"C:\Workspace\Geodata\MEP\Default.gdb\LidAg10m_Fill"
    )

In [63]:
# turn null values to zeros
out_raster = arcpy.ia.Con(
    in_conditional_raster="BufferedFlowlinesRaster",
    in_true_raster_or_constant="BufferedFlowlinesRaster",
    in_false_raster_or_constant=0,
    where_clause="Value IS NOT NULL"
)
out_raster.save(r"C:\Workspace\Geodata\MEP\Default.gdb\BufferedFlowlinesRaster_Null20")

In [64]:
# add burn layer to lidar elevation (since burn value is negative)
_o = "LidAg10B"
arcpy.ddd.Plus(os.path.join("LidAg10m_Fill"), "BufferedFlowlinesRaster_Null20", _o)

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\LidAg10B'>

## 4. Fill Sinks

In [65]:
# it is necessary to fill sinks again after buring in streams
# this will remove sinks behind road structures. 
# Fill sinks (again)
out_surface_raster = arcpy.sa.Fill(
    in_surface_raster="LidAg10B",
    z_limit=100
)
out_surface_raster.save(r"C:\Workspace\Geodata\MEP\Default.gdb\LidAg10BF")

## 5. Flow direction

In [66]:
# FLow direction 
_i = "LidAg10BF" # 10m resolution Burned and Filled
_o = "LidAg10BF_D8"
out_flow_direction_raster = arcpy.sa.FlowDirection(
    in_surface_raster=_i,
    force_flow="FORCE",
    out_drop_raster=None,
    flow_direction_type="D8"
)
out_flow_direction_raster.save(_o)

## 6. Flow accumulation

In [67]:
# Flow Accumulation
_i = "LidAg10BF_D8"
_o = "LidAg10BF_FlowAcc"
out_accumulation_raster = arcpy.sa.FlowAccumulation(
    in_flow_direction_raster=_i,
    in_weight_raster=None,
    data_type="FLOAT",
    flow_direction_type="D8"
)
out_accumulation_raster.save(_o)

## 7. Generate Bog Pour Points

In [69]:
# Zonal statistics to find the maximum flow accumulation inside each bog
out_raster = arcpy.sa.ZonalStatistics(
    in_zone_data=os.path.join(odr,"WMAbogsDRAFT2013_copy"),
    zone_field="FID",
    in_value_raster="LidAg10BF_FlowAcc",
    statistics_type="MAXIMUM",
    ignore_nodata="DATA",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_value=90,
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)
out_raster.save(r"C:\Workspace\Geodata\MEP\Default.gdb\Bog_Max_FlowAcc_ZS")

In [70]:
# Find points that equal of maximum value of flow accumulation inside each bog
outEqualTo = arcpy.ia.EqualTo("Bog_Max_FlowAcc_ZS", "LidAg10BF_FlowAcc")
outEqualTo.save("Bog_Max_FlowAcc_EQ")

In [71]:
# Set 0 as null value 
out_raster = arcpy.sa.SetNull(
    in_conditional_raster="Bog_Max_FlowAcc_EQ",
    in_false_raster_or_constant="Bog_Max_FlowAcc_EQ",
    where_clause="Value = 0"
)
out_raster.save(r"C:\Workspace\Geodata\MEP\Default.gdb\Bog_Max_FlowAcc_EQ_Null20")

In [72]:
# raster to point
arcpy.conversion.RasterToPoint(
    in_raster="Bog_Max_FlowAcc_EQ_Null20",
    out_point_features=r"C:\Workspace\Geodata\MEP\Default.gdb\BogPourPoints",
    raster_field="Value"
)

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\BogPourPoints'>

In [74]:
# I
arcpy.analysis.Identity(
    in_features="BogPourPoints",
    identity_features=os.path.join(odr,"WMAbogsDRAFT2013_copy"),
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\BogPourPoints_Ident",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\BogPourPoints_Ident'>

## 8. Delineate Watersheds

In [5]:
# Make a numpy array with the bog FID from Bog Pour Point Identity
inLyr = r"C:\Workspace\Geodata\MEP\Default.gdb\BogPourPoints_Ident"
idCol = "FID_WMAbogsDRAFT2013_copy"
_ = arcpy.da.FeatureClassToNumPyArray(inLyr,idCol)
# convert to data frame 
df_pourpoint = pd.DataFrame(_)
print(df_pourpoint.iloc[0:4,:])

   FID_WMAbogsDRAFT2013_copy
0                        546
1                        547
2                        977
3                        584


In [5]:
# select dataframe
df = df_pourpoint
df = df.loc[df['FID_WMAbogsDRAFT2013_copy'].isin([77,584,979])]
# create list to store output filenames
outfiles = [None]*len(df)
print(outfiles)

[None, None, None]


In [7]:
%%time 
# this cell took a Wall time: 4h 7min 26s for 982 watersheds
# loop through all rows of the cranberry bog pour points contained dataframe 
for i in range(len(df)):
    print(i)
    bogFID = df.iloc[i,0]

    print("{} = {}".format(idCol,bogFID))
    
    # clear the selection
    arcpy.management.SelectLayerByAttribute(
        in_layer_or_view="BogPourPoints_Ident",
        selection_type="CLEAR_SELECTION")
    
    # Select pour point by Attributes
    selection = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=inLyr,
        selection_type="NEW_SELECTION",
        where_clause="{} = {}".format(idCol,bogFID))
    
    """
    # save a temporary file with bog pour point
    tmpfile1 = os.path.join(tdr.name,"bogPt_{}".format(bogFID))
    bog = arcpy.management.CopyFeatures(
        in_features=selection,
        out_feature_class=tmpfile1)
    """                    
    # Delineate Watershed
    out_raster = arcpy.sa.Watershed(
        in_flow_direction_raster="LidAg10BF_D8",
        in_pour_point_data=selection,
        pour_point_field=idCol)
    '''
    # this block runs slower
    out_raster = arcpy.sa.Watershed(
        in_flow_direction_raster="LidAg10BF_D8",
        in_pour_point_data=tmpfile,
        pour_point_field="FID_WMAbog")
    '''
    
    # Save Watershed to outfile with name = bogFID
    tmpfile2 = os.path.join(odr,"bogWS{}.tif".format(bogFID))
    
    # create and save temporary output file
    out_raster.save(tmpfile2)
    
    outfile = os.path.join(odr,"bogWS_poly{}.shp".format(bogFID))
    arcpy.conversion.RasterToPolygon(
        in_raster=tmpfile2,
        out_polygon_features=outfile,
        simplify="SIMPLIFY",
        create_multipart_features="MULTIPLE_OUTER_PART",
        raster_field="Value")
    
    print(outfile)
    # Assign the name to the ith value of outfiles list
    outfiles[i] = outfile

0
FID_WMAbogsDRAFT2013_copy = 584
C:\Workspace\Geodata\MEP\outputs\bogWS_poly584.shp
1
FID_WMAbogsDRAFT2013_copy = 979
C:\Workspace\Geodata\MEP\outputs\bogWS_poly979.shp
2
FID_WMAbogsDRAFT2013_copy = 77
C:\Workspace\Geodata\MEP\outputs\bogWS_poly77.shp
Wall time: 46.5 s


In [3]:
# merge all bog watershed files and save to default geodatabase                       
arcpy.management.Merge(
    inputs=outfiles,
    output=r"C:\Workspace\Geodata\MEP\Default.gdb\bogWatersheds")

NameError: name 'outfiles' is not defined

## 9. Generate Output Table

In [6]:
arcpy.management.Dissolve(
    in_features="bogWatersheds",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\bogWatersheds_Dissolve",
    dissolve_field="gridcode",
    statistics_fields=None,
    multi_part="SINGLE_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\bogWatersheds_Dissolve'>

In [7]:
inLyr = "BogPourPoints_Ident"
joinLyr = "bogWatersheds_Dissolve"

# clear selections
_ = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=inLyr,
    selection_type="CLEAR_SELECTION"
)

_ = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=joinLyr,
    selection_type="CLEAR_SELECTION"
)


# add watershed areas to bog point data layer
_ = arcpy.management.JoinField(
    in_data=inLyr,
    in_field="FID_WMAbogsDRAFT2013_copy",
    join_table=joinLyr,
    join_field="gridcode",
    fields="Shape_Area",
    fm_option="NOT_USE_FM",
    field_mapping=None
)

# Save join output as new feature
arcpy.management.CopyFeatures(
    in_features=_,
    out_feature_class="BogPourPoints_WS_Ident")


<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\BogPourPoints_WS_Ident'>

In [11]:
inLyr = "BogPourPoints_WS_Ident"
selLyr = "MEP_SUBW_NAME" # select feature
outLyr = "BogPourPoints_Ident_MEP"
overlap = "WITHIN"

# Clear the current selection 
_ = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=inLyr,
    selection_type="CLEAR_SELECTION"
)

# Select all points that fall within with watersheds
_ = arcpy.management.SelectLayerByLocation(
    in_layer= _,
    overlap_type=overlap,
    select_features=selLyr,
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

# Save selection as new feature
arcpy.management.CopyFeatures(
    in_features=_,
    out_feature_class=outLyr)


<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\BogPourPoints_Ident_MEP'>

In [13]:
# Select only catchments that contain selected MEP bog pour points
inLyr = "bogWatersheds_Dissolve" # name of input layer to filter with query
inLyrField = "gridcode" # field to search for in input layer
qryLyr = "BogPourPoints_Ident_MEP" # query feature
qryLyrField = "FID_WMAbogsDRAFT2013_copy" # field from query layer from which to generate search list
outLyr = "bogWatersheds_Dissolve_MEP" # name of output layer

# Clear the current selection 
_ = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=inLyr,
    selection_type="CLEAR_SELECTION"
)
_ = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=qryLyr,
    selection_type="CLEAR_SELECTION"
)

# make a list of bog FIDs from query layer
_ = arcpy.da.FeatureClassToNumPyArray(qryLyr,qryLyrField)
_ = pd.DataFrame(_)
qry_list = [i[0] for i in _.values.tolist()]

# convert list into SQL 'where clause' query
qry = '"{0}" IN ({1})'.format(inLyrField,', '.join(map(str, qry_list)) or 'NULL')
print(qry)

"gridcode" IN (547, 977, 584, 167, 162, 582, 166, 583, 243, 764, 165, 631, 194, 629, 630, 164, 193, 585, 192, 191, 398, 189, 157, 190, 918, 397, 188, 123, 171, 117, 200, 396, 206, 395, 168, 919, 201, 662, 124, 202, 156, 154, 135, 170, 424, 136, 155, 169, 665, 664, 153, 740, 142, 545, 177, 134, 149, 122, 139, 146, 275, 976, 116, 494, 393, 272, 137, 84, 145, 159, 144, 966, 160, 121, 119, 199, 271, 118, 120, 659, 712, 392, 495, 147, 176, 128, 211, 125, 151, 484, 247, 140, 485, 152, 586, 174, 203, 876, 429, 141, 129, 150, 544, 426, 148, 138, 676, 704, 703, 197, 85, 427, 575, 883, 428, 196, 25, 581, 60, 874, 873, 195, 179, 350, 198, 875, 222, 223, 394, 86, 872, 26, 24, 877, 871, 163, 878, 957, 870, 879, 406, 54, 822, 563, 407, 19, 541, 641, 611, 354, 783, 684, 882, 881, 880, 253, 401, 332, 433, 540, 622, 610, 647, 648, 981, 520, 667, 414, 580, 612, 445, 823, 937, 604, 933, 888, 522, 603, 934, 956, 512, 953, 723, 511, 521, 35, 627, 526, 724, 525, 628, 527, 519, 786, 425, 767, 798, 607, 533, 

In [10]:
_ = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="bogWatersheds_Dissolve",
    selection_type="NEW_SELECTION",
    where_clause=qry,
    invert_where_clause=None
)

# Save selection as new feature
arcpy.management.CopyFeatures(
    in_features=_,
    out_feature_class=outLyr)

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\bogWatersheds_Dissolve_MEP'>

In [17]:
# Store SUBW_AREA as a variable
inFeature = "MEP_SUBW_NAME"
fieldName = "SUBW_NAME_Area"
# Create a new field
arcpy.management.AddField(inFeature, fieldName, "DOUBLE")
# Calculate field
arcpy.management.CalculateField(inFeature, fieldName, 
                                "!Shape_Area!",
                                "PYTHON3")

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\MEP_SUBW_NAME'>

In [18]:
# get MEP subwatershed names for each bog pour point
arcpy.analysis.Identity(
    in_features="BogPourPoints_Ident_MEP",
    identity_features="MEP_SUBW_NAME",
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\BogPourPoints_Ident_MEP_SUB",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\BogPourPoints_Ident_MEP_SUB'>

In [16]:
# Store topographic area as variable
inFeature = "bogWatersheds_Dissolve_MEP"
fieldName = "D8_ShedArea"
# Create a new field
arcpy.management.AddField(inFeature, fieldName, "DOUBLE")
 
# Calculate field
arcpy.management.CalculateField(inFeature, fieldName, 
                                "!Shape_Area!",
                                "PYTHON3")

# Store GW area as variable
inFeature = "bogWatersheds_Dissolve_MEP"
fieldName = "D8_ShedArea"
# Create a new field
arcpy.management.AddField(inFeature, fieldName, "DOUBLE")
 
# Calculate field
arcpy.management.CalculateField(inFeature, fieldName, 
                                "!Shape_Area!",
                                "PYTHON3")

# intersect 
identFeature = "MEP_SUBW_NAME"
_ = arcpy.analysis.Identity(
    in_features=inFeature,
    identity_features=identFeature,
    out_feature_class=r"C:\Workspace\Geodata\MEP\Default.gdb\bogWatersheds_Dissolve_MEP_SUB",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

# Store topographic area as variable
inFeature = _
fieldName = "GW_ShedArea"
# Create a new field
arcpy.management.AddField(inFeature, fieldName, "DOUBLE")
# Calculate field
arcpy.management.CalculateField(inFeature, fieldName, 
                                "!Shape_Area!",
                                "PYTHON3")

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\bogWatersheds_Dissolve_MEP_SUB'>

In [27]:
# Join the points table with the watersheds table.

# create copy of feature
arcpy.management.CopyFeatures(
    in_features="bogWatersheds_Dissolve_MEP_SUB",
    out_feature_class="copy")

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\copy'>

In [28]:
arcpy.management.AlterField(
    in_table=r"C:\Workspace\Geodata\MEP\Default.gdb\copy",
    field="FID_MEP_SUBW_NAME",
    new_field_name="FID_SUBW_shed",
    new_field_alias="FID_SUBW_shed",
    field_type="LONG",
    field_length=4,
    field_is_nullable="NULLABLE"
)

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\copy'>

In [29]:
arcpy.management.JoinField(
    in_data="copy",
    in_field="gridcode",
    join_table="BogPourPoints_Ident_MEP_SUB",
    join_field="FID_WMAbogsDRAFT2013_copy",
    fields="FID_MEP_SUBW_NAME;SUBW_NAME",
    fm_option="NOT_USE_FM",
    field_mapping=None
)

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\copy'>

In [ ]:
arcpy.management.CalculateField(
    in_table="copy",
    field="match",
    expression="!FID_SUBW_shed!==!FID_MEP_SUBW_NAME!",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [30]:
selection = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="copy",
    selection_type="NEW_SELECTION",
    where_clause="match = '1'",
    invert_where_clause=None
)

In [31]:
# Save selection as new feature
arcpy.management.CopyFeatures(
    in_features="copy",
    out_feature_class="bogWatersheds_Dissolve_MEP_SUB_Select")

<Result 'C:\\Workspace\\Geodata\\MEP\\Default.gdb\\bogWatersheds_Dissolve_MEP_SUB_Select'>

In [32]:
# Create new feature layer to hold extracts
arcpy.management.CopyFeatures(
    in_features="BogPourPoints_Ident_MEP_SUB",
    out_feature_class="BogPourPoints_Ident_MEP_SUB_Extr")

arcpy.sa.ExtractMultiValuesToPoints(
    in_point_features="BogPourPoints_Ident_MEP_SUB_Extr",
    in_rasters=r"C:\Workspace\Geodata\MEP\Default.gdb\bayes_krig_NOx bayes_krig_NOx;;C:\Workspace\Geodata\MEP\Default.gdb\lidar_le5pct lidar_le5pct",
    bilinear_interpolate_values="NONE"
)

<geoprocessing server result object object at 0x000001D213C46A20>

In [42]:
# Save selection as new feature
arcpy.management.CopyFeatures(
    in_features="BogPourPoints_Ident_MEP_SUB_Extr",
    out_feature_class="BogPourPoints_Ident_MEP_SUB_Extr_shed")

inLyr = "BogPourPoints_Ident_MEP_SUB_Extr"
joinLyr = "bogWatersheds_Dissolve_MEP_SUB_Select"

arcpy.management.JoinField(
    in_data=inLyr,
    in_field="FID_WMAbogsDRAFT2013_copy",
    join_table=joinLyr,
    join_field="gridcode",
    fields="SUBW_NAME;D8_ShedArea;GW_ShedArea",
    fm_option="NOT_USE_FM",
    field_mapping=None
)

<Result 'BogPourPoints_Ident_MEP_SUB_Extr'>

In [43]:
# Make a numpy array with the bog FID from Bog Pour Point Identity
inFeaturePath = "BogPourPoints_Ident_MEP_SUB_Extr_shed"
outTableName = "BogPourPoints_Ident_MEP_SUB_Extr_shed.xlsx"
outTablePath = odr
fn_arcpy_table_to_excel(inFeaturePath,outTablePath,outTableName)

# Appendix 
un-used snippets


arcpy.management.JoinField(
    in_data="BogPourPoints_Ident_MEP_SUB_Extr",
    in_field="FID_WMAbogsDRAFT2013_copy",
    join_table="MEP_SUBW_NAME",
    join_field="gridcode",
    fields="SUBW_NAME;D8_ShedArea;GW_ShedArea",
    fm_option="NOT_USE_FM",
    field_mapping=None
)